In [3]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-gpt4all/resolve/main/translated-gpt4all-code.json

In [4]:
import json

In [5]:
import random
from tqdm import tqdm

In [7]:
with open('translated-gpt4all-code.json') as fopen:
    code = json.load(fopen)

In [8]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [9]:
code[0]

{'prompt': ['Give me a Python function for a buffer that is smaller than the requested size.',
  '"Berikan saya fungsi Python untuk pengepam yang lebih kecil daripada saiz yang diminta."'],
 'response': ["You can define a buffer with a smaller size using the Python built-in bytearray() or by creating a custom array with the array module.\nFor example:\n\n```python\nbuffer = bytearray(256)\n``` or\n```python\nimport array\nbuffer = array.array('i', [0] * 256)\n```",
  "Anda boleh menentukan buffer dengan saiz yang lebih kecil menggunakan bytearray() bawaan Python atau dengan membuat array khusus dengan modul array. Contohnya:\n\n```python\nbuffer = bytearray(256)\n``` atau\n```python\nimport array\nbuffer = array.array('i', [0] * 256)\n```"],
 'source': 'unified_chip2'}

In [10]:
import malaya

language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

2023-07-25 14:04:45.785617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 14:04:45.863494: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 14:04:46.448336: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 14:04:46.448369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [11]:
language_detection.predict(code[0]['prompt'])

['other', 'standard-malay']

In [18]:
language_detection.predict(code[0]['response'])

['other', 'standard-malay']

In [19]:
data = []
for d in tqdm(code):
    try:
        r = d['response'][1]
        
        if language_detection.predict([r])[0] != 'standard-malay':
            continue
        
        en = d['prompt'][0]
        ms = d['prompt'][1]
            
        a = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': en,
            'output': r
        }
        data.append(a)
        
        if language_detection.predict([ms])[0] != 'standard-malay':
            continue
        
        a = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': ms,
            'output': r
        }
        data.append(a)
    
    except Exception as e:
        print(e)
        pass

100%|███████████████████████████████████| 70107/70107 [00:10<00:00, 6810.21it/s]


In [24]:
len(data)

104910

In [25]:
with open('prepared-gpt4all-code.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')